In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_baseline as file

In [2]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise = False

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37881 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-3d5c36680fb13c0f45a1d8bae99fcac5>,
 <Future: pending, key: main-4f528d85bbf46ea059898c4ba90aa41c>,
 <Future: pending, key: main-64dee34a5a3ffeb5bd8a268bc2700922>,
 <Future: pending, key: main-78f4d652ba700f822d99a480d933b8f6>,
 <Future: pending, key: main-1350a527f50fb382bd99d87e04f26f86>,
 <Future: pending, key: main-ea3976cff009c1f8dfe0289c6ccee3f6>,
 <Future: pending, key: main-2b81570ac0e8fe8edbe955f4aa45b304>,
 <Future: pending, key: main-9e4847b12dc0a64bc748b7c30d6f87d9>,
 <Future: pending, key: main-ef5b15474f9a5e448a4179b37f9893d5>,
 <Future: pending, key: main-2752958b0b65e305e4f50672840f8c55>,
 <Future: pending, key: main-aa15f009a71e4c042db815be0ce08a1b>,
 <Future: pending, key: main-b7c3b6177f04f612e085182befc7f3b8>,
 <Future: pending, key: main-dd79a32c2b6ed02304b692756ac2c47c>,
 <Future: pending, key: main-fa9970db8eb14610cbc6ee3c959ea743>,
 <Future: pending, key: main-b93b15e24e783fdabb4ac8ae830a29bf>,
 <Future: pending, key: main-b925646c837

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "simulation_baseline_ntrial={0}_n={1}_frontdoor={2}_minimise={3}.pt".format(ntrial,
                                                                                             n,
                                                                                             front_door,
                                                                                             minimise)
          )